## Loading the Data and Processing the Data

In [55]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

def load_and_filter_data(file_path, text_column):
    """
    Load the dataset from a CSV file and filter rows where the text column contains text.
    
    Args:
    file_path (str): Path to the CSV file.
    text_column (str): Name of the column containing text comments.
    
    Returns:
    pd.DataFrame: Filtered DataFrame with non-empty text comments.
    """
    data = pd.read_csv(file_path)
    filtered_data = data[data[text_column].notna() & (data[text_column] != '')]
    return filtered_data[['text']]  # Return only the text column

def preprocess_text(texts):
    """
    Preprocess the text data.
    
    Args:
    texts (pd.Series): Series containing text data.
    
    Returns:
    pd.Series: Preprocessed text data.
    """
    # Convert to lowercase and remove digits and non-alphabetic characters
    texts = texts.str.lower().str.replace(r'\d+', '', regex=True).str.replace(r'[^a-zA-Z\s]', '', regex=True)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    texts = texts.apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split() if word not in stop_words]))
    
    return texts

# Paths to your dataset files
file_path1 = 'dataset_tiktok-comments-scraper_2024-04-28_23-16-10-409.csv'
file_path2 = 'dataset_free-tiktok-scraper_2024-04-28_21-22-00-488.csv'

# Load and filter datasets
dataset1 = load_and_filter_data(file_path1, 'text')
dataset2 = load_and_filter_data(file_path2, 'text')

# Preprocess datasets
dataset1['text'] = preprocess_text(dataset1['text'])
dataset2['text'] = preprocess_text(dataset2['text'])

# Display the first few rows of the preprocessed datasets
print("Preprocessed Dataset 1:")
print(dataset1.head())
print("\nPreprocessed Dataset 2:")
print(dataset2.head())

# Print dataset shapes
print("Number of rows in Dataset 1:", dataset1.shape[0])
print("Number of rows in Dataset 2:", dataset2.shape[0])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ulugsali/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ulugsali/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/var/folders/ht/0hqtmhlj4mzdft95rl_k0_pr0000gn/T/ipykernel_3545/1914779020.py:19: DtypeWarning: Columns (23,26,29,99,135,139,150,151,159,163,166,171,174,175,178,179,183,186,187,191,194,195,199,206,207,210,211,214,219,222,226,234,235,238,239,242,246,247,250,255,262,278,282,283,286,291,299,302,303,306,314,323,326,330,338,342,346,358,359,362,363,366,370,371,374,375,378,379,382,383,386,391,394,398,399,402,419,422,426,430,431,434,445,449,450,453,454,457,461,465,469,473,477,481,485,489,493,497,498,501,522,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,7

Preprocessed Dataset 1:
                                                text
0                         maybe simpson real cartoon
1                             think designer purpose
2           waiting model back flip like one simpson
3  collab balenciaga yall look thing simpson put ...
4                                      video created

Preprocessed Dataset 2:
                                                text
0                    simpson v balenciaga fyp foryou
1  responder estatudoaquitudoo simpson x model th...
2  balenciaga x simpson balenciaga thesimpsons ca...
3  somebody think shoe balenciaga mudpit pfw fash...
4          balenciaga balenciaga balenciagacancelled
Number of rows in Dataset 1: 21136
Number of rows in Dataset 2: 2285


In [96]:
!pip install --upgrade openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [131]:
import pandas as pd
import requests

# Set up OpenAI API
api_key = "sk-proj-nzxRWSLF2BlxuIZDtD50T3BlbkFJEbLOkIlSA9KlwIJJuCQz"  # Set your OpenAI API key. Don't share this key and don't distribute a notebook that contains your key.

# Load your dataset
merged_dataset = pd.read_csv('Merged_Cleaned_Dataset.csv')

# Define function to perform sentiment analysis using OpenAI API
def perform_sentiment_analysis(text):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    data = {
        "model": "gpt-4-turbo-preview",  # Update with a supported model
        "prompt": text,
        "max_tokens": 1,
        "temperature": 0,  # Ensure deterministic output
        "logprobs": 10,    # Include log probabilities for each token
    }
    try:
        response = requests.post("https://api.openai.com/v1/completions", json=data, headers=headers)
        response.raise_for_status()  # Raise exception for HTTP errors
        response_json = response.json()
        print("Response JSON:", response_json)  # Debug print
        # Extract sentiment label from the response
        sentiment_label = response_json['choices'][0]['text'].strip()
        return sentiment_label
    except requests.exceptions.RequestException as e:
        print("Error performing API request:", e)
        return "Error"
    except (KeyError, IndexError) as e:
        print("Error processing API response:", e)
        return "Error"

# Apply sentiment analysis to each text instance in the dataset
merged_dataset['sentiment'] = merged_dataset['text'].apply(perform_sentiment_analysis)

# Save the updated dataset with sentiment labels
merged_dataset.to_csv('Labeled_Dataset_Sentiment.csv', index=False)

Error performing API request: 404 Client Error: Not Found for url: https://api.openai.com/v1/completions
Error performing API request: 404 Client Error: Not Found for url: https://api.openai.com/v1/completions
Error performing API request: 404 Client Error: Not Found for url: https://api.openai.com/v1/completions
Error performing API request: 404 Client Error: Not Found for url: https://api.openai.com/v1/completions
Error performing API request: 404 Client Error: Not Found for url: https://api.openai.com/v1/completions
Error performing API request: 404 Client Error: Not Found for url: https://api.openai.com/v1/completions
Error performing API request: 404 Client Error: Not Found for url: https://api.openai.com/v1/completions


KeyboardInterrupt: 